In [1]:
%%capture --no-display output

import os, time

from pathlib import Path

import numpy as np
import tensorflow as tf
import stable_baselines.common.tf_util as tf_util
from stable_baselines.composeNet import SACComposenet
from stable_baselines.composeNet.policies import ComposenetPolicy

from env_script.env_mujoco import JacoMujocoEnv;


model_path = "models_baseline/"

args = {}
args['task'] = 'picking'
args['subgoal_obs'] = False
args['rulebased_subgoal'] = True

env = JacoMujocoEnv(**args)

In [2]:
%%capture --no-display output

use_embedding = True

model = SACComposenet(policy=ComposenetPolicy,
                        env=env,
                        use_embedding=use_embedding)


prim_name = 'reaching_pick'
policy_zip_path = model_path+'reaching/policy.zip'
model.setup_skills(name=prim_name, obs_idx=[1,2,3,4,5,6, 17,18,19,20,21,22],
                    obs_relativity={'subtract':{'ref':[17,18,19,20,21,22],'tar':[1,2,3,4,5,6]}},
                    loaded_policy=SACComposenet._load_from_file(policy_zip_path))

prim_name = 'grasping'
policy_zip_path = model_path+'grasping/policy.zip'
model.setup_skills(name=prim_name, obs_idx=[0, 1,2,3,4,5,6, 7, 8,9,10],
                    obs_relativity={},
                    loaded_policy=SACComposenet._load_from_file(policy_zip_path))

model_dict = {'verbose': 1, 'seed': 0, 'gamma': 0.99, 'learning_starts':0, 
            'ent_coef': 1e-4, 'batch_size': 8, 'noptepochs': 4, 'n_steps': 128}
model.__dict__.update(model_dict)
model.setup_model()

# composenet_path = model_path + "ComposeNet/picking_emb_trained_at_2022_5_2_17:9_0/policy_440000.zip"
composenet_path = model_path + "ComposeNet/picking_emb_trained_at_2022_5_2_17:9_0/policy_2420000.zip"
loaded_policy = SACComposenet._load_from_file(composenet_path)
model.load_parameters(composenet_path, exact_match=False)


In [3]:
obs = env.reset()

sample goal from None


In [4]:

action, _ = model.predict([obs])
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)

print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))
next_obs, reward, done, _ = env.step(action[0])
print(reward)
idx = 0

[-28.91345  -36.968925 -38.647606  -4.929733  21.904453  38.151794
 -29.482439]
[7.389056  7.389056  7.389056  2.3655016 7.389056  7.389056  7.389056 ]
[-27.400043  -23.867613  -33.86832    -3.0458875  15.39091    36.534893
 -25.847994 ]
[-1.0, -1.0, -1.0, -0.9954874216759722, 0.9999999999999144, 1.0, -1.0]
0.014971267534871449


In [105]:
# idx = 0
feed_dict = {
        model.observations_ph: [obs],
        model.actions_ph: action,
        # model.rewards_ph: [[reward]],
        model.rewards_ph: [[-1]],
        model.terminals_ph: [[done]],
        model.learning_rate_ph: 1e-3
    }
for _ in range(100):
    idx += 1
    output = model.sess.run(model.step_ops, feed_dict)
print(output)

[-1.4775512, 1.3728396e-10, 1.5987212e-10, 9.266643e-09, array([[1.1326133]], dtype=float32), array([[1.1325788]], dtype=float32), array([[1.4776874]], dtype=float32), array([73.765656], dtype=float32), 23.932568, None, None]


In [14]:
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

0
[-28.91345  -36.968925 -38.647606  -4.929733  21.904453  38.151794
 -29.482439]
[7.389056  7.389056  7.389056  2.3655016 7.389056  7.389056  7.389056 ]
[-25.584692 -25.737532 -45.58187   -5.731157   8.344255  38.56462
 -10.9783  ]
[-1.0, -1.0, -1.0, -0.999978961931814, 0.9999998869418751, 1.0, -0.9999999994173608]
[[0.87680876]] [[0.9602536]] [[2.163801]] [[2.154138]] [72.69124]


In [22]:
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

3
[-28.871334 -36.89931  -38.499943  -3.921737  21.872208  38.29764
 -29.563982]
[7.389056  7.389056  7.389056  3.6077366 7.389056  7.389056  7.389056 ]
[-32.578884 -37.93266  -41.16772   -6.191561  42.334827  48.088306
 -25.938114]
[-1.0, -1.0, -1.0, -0.9999916226544373, 1.0, 1.0, -1.0]
[[1.5144858]] [[1.5206213]] [[2.0876346]] [[2.154138]] [68.48819]


In [49]:
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

6
[-28.844912  -36.865448  -38.40487    -3.2271166  21.85876    38.40322
 -29.615131 ]
[7.389056 7.389056 7.389056 4.51464  7.389056 7.389056 7.389056]
[-20.141628   -45.516113   -54.724472    -0.17024994  23.502071
  41.26575    -41.67757   ]
[-1.0, -1.0, -1.0, -0.1686238882969094, 1.0, 1.0, -1.0]
[[2.3516726]] [[2.2730892]] [[2.0258908]] [[2.154138]] [70.65289]


In [51]:
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

9
[-28.842438 -36.884197 -38.359547  -2.847522  21.734457  38.462708
 -29.667517]
[7.389056  7.389056  7.389056  6.6375656 7.389056  7.389056  7.389056 ]
[-25.62601   -46.407776  -49.25657    -5.4304056  10.53967    42.851933
 -32.01203  ]
[-1.0, -1.0, -1.0, -0.9999616088367568, 0.9999999985991587, 1.0, -1.0]
[[2.5949004]] [[2.5036504]] [[1.9936068]] [[2.154138]] [73.35779]


In [71]:
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

12
[-28.824526  -36.883335  -38.295902   -2.4107032  21.58131    38.527267
 -29.726261 ]
[7.389056 7.389056 7.389056 7.389056 7.389056 7.389056 7.389056]
[-35.73266   -43.041817  -38.476307   -6.3291836  10.660076   36.491108
 -34.364277 ]
[-1.0, -1.0, -1.0, -0.9999936383523981, 0.9999999988989539, 1.0, -1.0]
[[2.583111]] [[2.568245]] [[2.0011048]] [[2.154138]] [60.381836]


In [73]:
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

15
[-28.809935  -36.881042  -38.246834   -2.0730898  21.468187   38.57715
 -29.77017  ]
[7.389056 7.389056 7.389056 7.389056 7.389056 7.389056 7.389056]
[-30.027296  -35.60517   -56.63739     5.0740147  23.765175   33.449757
 -34.73978  ]
[-1.0, -1.0, -1.0, 0.9999216967382878, 1.0, 1.0, -1.0]
[[2.8665302]] [[2.9639912]] [[2.0330744]] [[2.154138]] [69.23624]


In [87]:
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

18
[-28.795012  -36.87482   -38.203445   -1.7719867  21.381544   38.621803
 -29.80552  ]
[7.389056 7.389056 7.389056 7.389056 7.389056 7.389056 7.389056]
[-24.812721 -40.679092 -29.87246    8.189885  12.457787  21.52821
 -47.728508]
[-1.0, -1.0, -1.0, 0.9999998460477235, 0.9999999999697773, 1.0, -1.0]
[[2.5870175]] [[2.7208703]] [[2.0695634]] [[2.154138]] [62.62728]


In [89]:
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

118
[-26.590157 -37.23576  -37.585728  12.967801  20.839033  41.748642
 -31.24032 ]
[7.389056 7.389056 7.389056 7.389056 7.389056 7.389056 7.389056]
[-30.120619 -36.05077  -39.311176  15.622508  20.34321   44.522488
 -36.044827]
[-1.0, -1.0, -1.0, 0.9999999999999462, 1.0, 1.0, -1.0]
[[2.6390877]] [[2.6579528]] [[2.596641]] [[2.154138]] [74.26238]


In [91]:
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

218
[-24.466154 -37.924644 -37.890106  18.821583  20.525024  43.37168
 -32.050728]
[7.389056 7.389056 7.389056 7.389056 7.389056 7.389056 7.389056]
[-29.671314  -38.950924  -40.324818    2.4193554   9.056836   54.296722
 -25.756971 ]
[-1.0, -1.0, -1.0, 0.9842898720438272, 0.9999999728129503, 1.0, -1.0]
[[2.6366394]] [[2.6550474]] [[2.63421]] [[2.154138]] [71.3647]


In [97]:
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

318
[-26.454084 -38.517128 -38.240726  22.718016  22.553564  44.588856
 -32.65582 ]
[7.389056 7.389056 7.389056 7.389056 7.389056 7.389056 7.389056]
[-28.343935 -36.221813 -49.777657  31.249178  22.963732  49.59545
 -34.363174]
[-1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0]
[[2.6406775]] [[2.6592236]] [[2.6314058]] [[2.154138]] [74.12714]


In [100]:
# After reward from 0.014 -> 1
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

100
[-26.532717 -38.729015 -38.38537   23.964941  22.329811  44.99713
 -30.030699]
[7.389056 7.389056 7.389056 7.389056 7.389056 7.389056 7.389056]
[-24.120192 -32.904034 -29.170898  17.61282   30.35894   46.611618
 -21.324055]
[-1.0, -1.0, -1.0, 0.999999999999999, 1.0, 1.0, -1.0]
[[3.840393]] [[3.854195]] [[3.8404708]] [[2.154138]] [71.03593]


In [102]:
# After reward from 0.014 -> 1
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

200
[-26.58019  -38.85143  -38.467533  23.975801  25.020857  45.062447
 -27.074781]
[7.389056 7.389056 7.389056 7.389056 7.389056 7.389056 7.389056]
[-30.62355  -38.9067   -50.100918  24.779272  21.287333  37.440277
 -21.2266  ]
[-1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0]
[[3.8425891]] [[3.85951]] [[3.835316]] [[2.154138]] [71.86378]


In [104]:
# After reward from 1 -> -1
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

100
[-26.579065 -38.84868  -38.465572  23.9854    24.915907  45.063175
 -27.068155]
[7.389056 7.389056 7.389056 7.389056 7.389056 7.389056 7.389056]
[-31.065365 -48.83209  -42.24901   19.653442  22.289143  52.931988
 -37.385513]
[-1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0]
[[1.4876363]] [[1.4898908]] [[1.4549265]] [[2.154138]] [73.764786]


In [106]:
# After reward from 1 -> -1
print(idx)
feed_dict = {model.policy_tf.obs_ph: [obs]}
act_mu, std, pi = model.sess.run([model.policy_tf.mu_, model.policy_tf.std, model.policy_tf.pi_], feed_dict)
print(act_mu[0])
print(std[0])
print(pi[0])
print(list(map(np.math.tanh, pi[0])))

feed_dict = {model.policy_tf.obs_ph: [obs], model.actions_ph: [list(map(np.math.tanh, pi[0]))]}
qf1, qf2, value, value_target, logp_pi = model.sess.run([model.policy_tf.qf1[0], model.policy_tf.qf2[0], model.policy_tf.value_fn[0], model.value_target, model.policy_tf.logp_pi], feed_dict)
print(qf1, qf2, value, value_target, logp_pi)

200
[-26.579082 -38.846535 -38.463707  23.985037  24.90554   45.061413
 -23.69035 ]
[7.389056 7.389056 7.389056 7.389056 7.389056 7.389056 7.389056]
[-22.500452 -44.701054 -36.644222  23.706997  22.508974  46.718807
 -31.863733]
[-1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0]
[[1.4849249]] [[1.4853393]] [[1.4776785]] [[2.154138]] [68.43728]
